In [1]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
#client = Socrata("data.cityofnewyork.us", None)

# Example authenticated client (needed for non-public datasets):
client = Socrata("data.cityofnewyork.us",
                  "", # add your token
                  username="", # add your username
                  password="") # add your password

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results1 = client.get("66ae-7zpy", final_call_type="CARD", limit=200000)

Check API is working

In [2]:
results1[0]

{'borough': 'BRONX',
 'cad_incident_id': '130010002',
 'citycouncildistrict': '8',
 'communitydistrict': '201',
 'communityschooldistrict': '7',
 'congressionaldistrict': '15',
 'dispatch_response_seconds_qy': '59',
 'final_call_type': 'CARD',
 'final_severity_level_code': '3',
 'first_activation_datetime': '2013-01-01T00:02:08.000',
 'first_assignment_datetime': '2013-01-01T00:01:18.000',
 'first_on_scene_datetime': '2013-01-01T00:14:30.000',
 'held_indicator': 'N',
 'incident_close_datetime': '2013-01-01T00:55:34.000',
 'incident_datetime': '2013-01-01T00:00:19.000',
 'incident_dispatch_area': 'B1',
 'incident_disposition_code': '93',
 'incident_response_seconds_qy': '851',
 'incident_travel_tm_seconds_qy': '792',
 'initial_call_type': 'CARD',
 'initial_severity_level_code': '3',
 'policeprecinct': '40',
 'reopen_indicator': 'N',
 'special_event_indicator': 'N',
 'standby_indicator': 'N',
 'transfer_indicator': 'N',
 'valid_dispatch_rspns_time_indc': 'Y',
 'valid_incident_rspns_time_

It's working!

Create a table with the results

In [4]:
card_list = []

for row in results1:
    
    rowdict = {}
    
    try:
        
        rowdict['zipcode'] = row['zipcode']
        rowdict['borough'] = row['borough']
        rowdict['incident_datetime'] = row['incident_datetime']
        rowdict['first_assignment_datetime'] = row['first_assignment_datetime']
        rowdict['incident_travel_tm_seconds_qy'] = row['incident_travel_tm_seconds_qy']
        rowdict['incident_response_seconds_qy'] = row['incident_response_seconds_qy']
        rowdict['first_on_scene_datetime'] = row['first_on_scene_datetime']
        rowdict['first_to_hosp_datetime'] = row['first_to_hosp_datetime']
        rowdict['first_hosp_arrival_datetime'] = row['first_hosp_arrival_datetime']
        rowdict['dispatch_response_seconds_qy'] = row['dispatch_response_seconds_qy']
        rowdict['incident_disposition_code'] = row['incident_disposition_code']
        rowdict['final_call_type'] = row['final_call_type']

    #rowdict[''] = row['']

    except:
        
        continue
        
    card_list.append(rowdict)
    
card_df = pd.DataFrame.from_records(card_list)


View the table

In [6]:
card_df.head()

,borough,dispatch_response_seconds_qy,final_call_type,first_assignment_datetime,first_hosp_arrival_datetime,first_on_scene_datetime,first_to_hosp_datetime,incident_datetime,incident_disposition_code,incident_response_seconds_qy,incident_travel_tm_seconds_qy,zipcode
0,BROOKLYN,19,CARD,2013-01-01T00:22:56.000,2013-01-01T01:08:51.000,2013-01-01T00:28:54.000,2013-01-01T00:52:03.000,2013-01-01T00:22:37.000,82,377,358,11205
1,RICHMOND / STATEN ISLAND,10,CARD,2013-01-01T00:29:35.000,2013-01-01T01:17:30.000,2013-01-01T00:36:54.000,2013-01-01T01:01:06.000,2013-01-01T00:29:25.000,82,449,439,10314
2,MANHATTAN,93,CARD,2013-01-01T00:36:22.000,2013-01-01T01:13:06.000,2013-01-01T00:43:52.000,2013-01-01T01:08:04.000,2013-01-01T00:34:49.000,82,543,450,10065
3,BROOKLYN,8,CARD,2013-01-01T00:37:33.000,2013-01-01T01:03:39.000,2013-01-01T00:39:14.000,2013-01-01T00:58:19.000,2013-01-01T00:37:25.000,82,109,101,11206
4,QUEENS,18,CARD,2013-01-01T00:39:40.000,2013-01-01T01:22:42.000,2013-01-01T00:46:43.000,2013-01-01T01:15:12.000,2013-01-01T00:39:22.000,82,441,423,11420


Clean numbers and formats

In [7]:
card_df['incident_response_seconds_qy'] = card_df['incident_response_seconds_qy'].astype(int)

In [21]:
zipcodes_data_card = card_df.groupby(['zipcode', 'borough']).incident_response_seconds_qy.mean()
zipcodes_data_card.head()

zipcode  borough  
10000    MANHATTAN    709.000000
10001    MANHATTAN    462.530647
10002    MANHATTAN    423.896980
10003    MANHATTAN    395.040365
10004    MANHATTAN    433.996633
Name: incident_response_seconds_qy, dtype: float64

In [22]:
zipcodes_data_card.dtypes

dtype('float64')

In [23]:
zipcodes_data_card = zipcodes_data_card.reset_index()
zipcodes_data_card.head()

,zipcode,borough,incident_response_seconds_qy
0,10000,MANHATTAN,709.000000
1,10001,MANHATTAN,462.530647
2,10002,MANHATTAN,423.896980
3,10003,MANHATTAN,395.040365
4,10004,MANHATTAN,433.996633


In [24]:
zipcodes_data_card.dtypes

zipcode                          object
borough                          object
incident_response_seconds_qy    float64
dtype: object

In [25]:
zipcodes_data_card['incident_response_min_qy'] = round((zipcodes_data_card['incident_response_seconds_qy'])/60,2)
zipcodes_data_card.head()

,zipcode,borough,incident_response_seconds_qy,incident_response_min_qy
0,10000,MANHATTAN,709.000000,11.82
1,10001,MANHATTAN,462.530647,7.71
2,10002,MANHATTAN,423.896980,7.06
3,10003,MANHATTAN,395.040365,6.58
4,10004,MANHATTAN,433.996633,7.23


In [33]:
zipcodes_data_card.sort_values('incident_response_min_qy', ascending=False).head(30)

,zipcode,borough,incident_response_seconds_qy,incident_response_min_qy
53,10153,MANHATTAN,811.285714,13.52
206,11697,QUEENS,752.275862,12.54
0,10000,MANHATTAN,709.000000,11.82
86,10464,BRONX,670.178082,11.17
98,10803,BRONX,668.333333,11.14
45,10103,MANHATTAN,649.000000,10.82
117,11208,QUEENS,635.000000,10.58
149,11241,BROOKLYN,617.500000,10.29
40,10044,MANHATTAN,611.166667,10.19
50,10123,MANHATTAN,601.333333,10.02


In [34]:
zipcodes_data_card.to_csv('card.csv', index=False)